## 1. Data Preparation

### 1.1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV , KFold
from sklearn.metrics import mean_squared_error 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge , ElasticNet , Lasso
from sklearn.ensemble import RandomForestRegressor  , AdaBoostRegressor 
from sklearn.preprocessing import MinMaxScaler

- Import data, check for comlumns and variables
### 1.2. Load and Clean Data
Load the dataset, inspect its structure, and perform initial cleaning. We drop the original 'Price' column (in Toman) and use 'Price(USD)' for our analysis, renaming it to 'Price'.

In [ ]:
houses = pd.read_csv("./data/housePrice.csv")
houses = houses.drop('Price', axis=1)
houses = houses.rename(columns={'Price(USD)': 'Price'})
houses.info()
houses.head()

- Since the Address column has some missing data in certain rows, we will drop these rows.

In [ ]:
houses = houses.dropna(subset=['Address'])

- Check the header again

In [ ]:
houses.info()

- Check the correlation between variables and the price variable before converting them into categories (actually this is feature engineering xD)

In [ ]:
# houses['Price'] = np.log1p(houses['Price'])
# houses['Area'] = np.log1p(pd.to_numeric(houses['Area'], errors='coerce'))
houses['Area'] = pd.to_numeric(houses['Area'], errors='coerce')
houses = houses.dropna(subset=['Area'])
houses['Room'] = houses['Room'].astype('int64')
houses['Area_per_Room'] = houses['Area'] / (houses['Room']+1)
houses['Area_per_Room'] = houses['Area_per_Room'].replace([np.inf, -np.inf], np.nan)
address_avg_price = houses.groupby('Address')['Price'].mean()
houses['Avg_Price_by_Address'] = houses['Address'].map(address_avg_price)
houses['Price_per_m2'] = houses['Price'] / houses['Area']
address_avg_price_per_m2 = houses.groupby('Address')['Price_per_m2'].mean()
# houses['Avg_Price_per_m2_by_Address'] = houses['Address'].map(address_avg_price_per_m2)
houses = houses.drop('Price_per_m2', axis=1)

# houses['Parking_Warehouse'] = (houses['Parking'] == True) & (houses['Warehouse'] == True)
# houses['Parking_Elevator'] = (houses['Parking'] == True) & (houses['Elevator'] == True)
# houses['Elevator_Warehouse'] = (houses['Elevator'] == True) & (houses['Warehouse'] == True)
# houses['All_Amenities'] = (houses['Parking'] == True) & (houses['Warehouse'] == True) & (houses['Elevator'] == True)

numeric_houses = houses.select_dtypes(include=[np.number, bool]) 
corr_matrix = numeric_houses.corr()
corr_matrix["Price"].sort_values(ascending=False)

- That said, the data is now complete. HOWEVER, we have not yet categorized the data into the two main types: Numerical and Categorical. Now, we will do that.

In [ ]:
# Convert specified columns to categorical data type
houses['Room'] = houses['Room'].astype('category')
houses['Parking'] = houses['Parking'].astype('category')
houses['Warehouse'] = houses['Warehouse'].astype('category')
houses['Elevator'] = houses['Elevator'].astype('category')
# Verify the changes
print("Data types after conversion:")
print(houses.dtypes)
houses.info()

### 1.3. Outlier Removal
To prevent extreme values from skewing the analysis, we remove the top and bottom 1% of properties based on price.

In [ ]:
# Calculate 1st and 99th percentiles
lower_percentile = houses['Price'].quantile(0.01)
upper_percentile = houses['Price'].quantile(0.99) # !!

price_min = lower_percentile
price_max = upper_percentile

print(price_max, price_min)

print(f"1st percentile (1% lowest): ${lower_percentile:.2f}")
print(f"99th percentile (1% highest): ${upper_percentile:.2f}")

# Remove outliers
houses_before = len(houses)
houses = houses[(houses['Price'] >= lower_percentile) & (houses['Price'] <= upper_percentile)]
houses_after = len(houses)

print(f"\nDataset size before removing outliers: {houses_before}")
print(f"Dataset size after removing outliers: {houses_after}")
print(f"Removed {houses_before - houses_after} outliers ({((houses_before - houses_after) / houses_before * 100):.1f}%)")

## 2. Data Preparation


- first 10 rows

In [ ]:
houses.head(10)

- This part was already carried out in quite detail in the file `houses_price.ipynb`; you can refer to that notebook for more information.

- One-hot encoding

In [ ]:
# One-hot encoding for Address column
address_encoded = pd.get_dummies(houses['Address'], prefix='Address')

# Concatenate the one-hot encoded columns with the original dataframe
houses_encoded = pd.concat([houses, address_encoded], axis=1)

# Drop the original Address column since we now have encoded versions
houses_encoded = houses_encoded.drop('Address', axis=1)

print(f"Original dataset shape: {houses.shape}")
print(f"Dataset shape after one-hot encoding: {houses_encoded.shape}")
print(f"Added {address_encoded.shape[1]} new columns for Address categories")

# Display the new column names
print(f"\nNew Address columns created:")
print(list(address_encoded.columns))

# Update our main dataframe
houses = houses_encoded
houses.head()

- Transform cols

In [ ]:

X = houses.drop('Price', axis=1)  
y = houses['Price']              
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns:")
print(list(X.columns))


X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,     
    random_state=42     
)

scaler_X = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

scaler_y = MinMaxScaler()

y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)

y_train_scaled = scaler_y.fit_transform(y_train_reshaped)

y_test_scaled = scaler_y.transform(y_test_reshaped)

y_train = pd.Series(y_train_scaled.flatten(), name=y_train.name, index=y_train.index)
y_test = pd.Series(y_test_scaled.flatten(), name=y_test.name, index=y_test.index)

X_train.head()

## 3. Model training

- Split into test_set and train_set

In [ ]:

print(f"\nTraining set:")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

print(f"\nTesting set:")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

print(f"\nData split summary:")
print(f"Total samples: {len(houses)}")
print(f"Training samples: {len(X_train)} ({len(X_train)/len(houses)*100:.1f}%)")
print(f"Testing samples: {len(X_test)} ({len(X_test)/len(houses)*100:.1f}%)")

- try with models

- just convert function

In [ ]:
price_min = scaler_y.data_min_[0]
price_max = scaler_y.data_max_[0]
def convert_scaled_to_original_price(scaled_value, price_min=None, price_max=None):
    """
    Convert scaled price value back to original USD price
    
    Parameters:
    scaled_value: float (0-1) - giá trị đã được MinMaxScaler
    price_min: float - giá trị min của Price sau khi log1p 
    price_max: float - giá trị max của Price sau khi log1p 
    
    Returns:
    float - giá trị USD gốc
    """
    
    # Bước 1: Unscale từ [0,1] về log scale
    log_price = scaled_value * (price_max - price_min) + price_min
    
    # Bước 2: Inverse log transform (expm1 là ngược lại của log1p)
    # original_price = np.expm1(log_price)
    original_price = log_price
    
    return original_price


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
import pandas as pd

models = {
    'Ridge Regression': Ridge(random_state=42),
    'Lasso Regression': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'K-Neighbors': KNeighborsRegressor(),
    'AdaBoost': AdaBoostRegressor(random_state=42)
}

# Store results
results = {}

print("Running K-Fold Cross Validation (k=10) for each model...")
print("=" * 60)

# Test each model with k-fold cross validation
for model_name, model in models.items():
    print(f"\nTesting {model_name}...")
    
    # Perform 10-fold cross validation
    # Using neg_root_mean_squared_error (negative RMSE)
    cv_scores = -cross_val_score(
        model, X_train, y_train,
        scoring="neg_root_mean_squared_error",
        cv=10,
        n_jobs=-1  # Use all available processors
    )
    
    # Store results
    results[model_name] = cv_scores
    
    # Display statistics
    score_stats = pd.Series(cv_scores).describe()
    print(f"Cross-validation RMSE scores for {model_name}:")
    print(score_stats)
    print(f"Mean RMSE: {cv_scores.mean():.6f} (+/- {cv_scores.std() * 2:.6f})")
    
    # Convert to original USD
    mean_rmse_usd = convert_scaled_to_original_price(cv_scores.mean(), price_min, price_max)
    print(f"Mean RMSE in original USD: ${mean_rmse_usd:,.2f}")

print("\n" + "=" * 60)
print("SUMMARY OF ALL MODELS")
print("=" * 60)

# Create summary comparison
summary_data = []
for model_name, scores in results.items():
    mean_rmse = scores.mean()
    mean_rmse_usd = convert_scaled_to_original_price(mean_rmse, price_min, price_max)
    
    summary_data.append({
        'Model': model_name,
        'Mean RMSE': mean_rmse,
        'Mean RMSE (USD)': mean_rmse_usd,
        'Std RMSE': scores.std(),
        'Min RMSE': scores.min(),
        'Max RMSE': scores.max()
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Mean RMSE')

# Format the USD column for better display
summary_df_display = summary_df.copy()
summary_df_display['Mean RMSE (USD)'] = summary_df_display['Mean RMSE (USD)'].apply(lambda x: f"${x:,.2f}")

print(summary_df_display[['Model', 'Mean RMSE', 'Mean RMSE (USD)', 'Std RMSE']].to_string(index=False))

# Find the best model
best_model_name = summary_df.iloc[0]['Model']
best_rmse = summary_df.iloc[0]['Mean RMSE']
best_rmse_usd = summary_df.iloc[0]['Mean RMSE (USD)']

print(f"\nBest performing model: {best_model_name}")
print(f"Best RMSE: {best_rmse:.6f}")
print(f"Best RMSE in original USD: ${best_rmse_usd:,.2f}")

In [ ]:
best_model = RandomForestRegressor(random_state=42)
best_model.fit(X_train, y_train)

y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

rmse_train_usd = convert_scaled_to_original_price(rmse_train, price_min, price_max)
rmse_test_usd = convert_scaled_to_original_price(rmse_test, price_min, price_max)

print("Random Forest Model Performance:")
print("=" * 50)
print(f"Training Set:")
print(f"  R² Score: {r2_train:.6f}")
print(f"  RMSE: {rmse_train:.6f}")
print(f"  RMSE (USD): ${rmse_train_usd:,.2f}")

print(f"\nTest Set:")
print(f"  R² Score: {r2_test:.6f}")
print(f"  RMSE: {rmse_test:.6f}")
print(f"  RMSE (USD): ${rmse_test_usd:,.2f}")

print(f"\nModel Comparison:")
print(f"  Difference in R² (Train - Test): {r2_train - r2_test:.6f}")
print(f"  Difference in RMSE (Test - Train): {rmse_test - rmse_train:.6f}")

if r2_train - r2_test > 0.1:
    print("Potential overfitting detected (R² difference > 0.1)")
else:
    print("Good generalization performance")

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

# Define parameter distributions for RandomizedSearchCV
param_distribs = {
    'n_estimators': randint(low=50, high=300),
    'max_features': ['sqrt', 'log2', None],
    'max_depth': randint(low=3, high=20),
    'min_samples_split': randint(low=2, high=20),
    'min_samples_leaf': randint(low=1, high=10),
    'bootstrap': [True, False]
}

# Create RandomizedSearchCV
rnd_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_distribs,
    n_iter=100,  # Number of parameter settings sampled
    cv=5,        # 5-fold cross validation
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=-1    # Use all available processors
)

print("Starting Randomized Search CV...")
start_time = time.time()

# Fit the randomized search
rnd_search.fit(X_train, y_train)

end_time = time.time()
print(f"Randomized Search completed in {end_time - start_time:.2f} seconds")

# Get the best parameters and score
print("\nBest parameters found:")
print(rnd_search.best_params_)
print(f"\nBest cross-validation RMSE: {-rnd_search.best_score_:.6f}")
print(f"Best RMSE in USD: ${convert_scaled_to_original_price(-rnd_search.best_score_, price_min, price_max):,.2f}")

# Train the best model on full training set and evaluate
best_rf_model = rnd_search.best_estimator_
y_train_pred_best = best_rf_model.predict(X_train)
y_test_pred_best = best_rf_model.predict(X_test)

# Calculate performance metrics
r2_train_best = r2_score(y_train, y_train_pred_best)
r2_test_best = r2_score(y_test, y_test_pred_best)
rmse_train_best = np.sqrt(mean_squared_error(y_train, y_train_pred_best))
rmse_test_best = np.sqrt(mean_squared_error(y_test, y_test_pred_best))

print(f"\nOptimized Random Forest Performance:")
print(f"Training R²: {r2_train_best:.6f}")
print(f"Test R²: {r2_test_best:.6f}")
print(f"Training RMSE: {rmse_train_best:.6f} (${convert_scaled_to_original_price(rmse_train_best, price_min, price_max):,.2f})")
print(f"Test RMSE: {rmse_test_best:.6f} (${convert_scaled_to_original_price(rmse_test_best, price_min, price_max):,.2f})")

# Compare with original model
print(f"\nComparison with original Random Forest:")
print(f"Original Test RMSE: {rmse_test:.6f} vs Optimized: {rmse_test_best:.6f}")
print(f"Improvement: {((rmse_test - rmse_test_best) / rmse_test * 100):.2f}%")

## 4. EDA test

In [ ]:
print(price_min)
print(price_max)

# Calculate percentage errors for the optimized Random Forest model
y_test_original = scaler_y.inverse_transform(y_test.values.reshape(-1, 1)).flatten()
y_test_pred_original = scaler_y.inverse_transform(y_test_pred_best.reshape(-1, 1)).flatten()

print("Sample of original prices:")
print(y_test_original[:5])
print("\nSample of predicted prices:")
print(y_test_pred_original[:5])

# Calculate percentage error
percentage_errors = np.abs((y_test_pred_original - y_test_original) / y_test_original) * 100

# Identify predictions with error > 20%
high_error_mask = percentage_errors > 20
high_error_indices = np.where(high_error_mask)[0]

print(f"Total test samples: {len(y_test)}")
print(f"Predictions with error > 20%: {np.sum(high_error_mask)} ({np.sum(high_error_mask)/len(y_test)*100:.1f}%)")
print(f"Mean percentage error: {percentage_errors.mean():.2f}%")
print(f"Median percentage error: {np.median(percentage_errors):.2f}%")

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Scatter plot: Actual vs Predicted prices
ax1 = axes[0, 0]
scatter = ax1.scatter(y_test_original, y_test_pred_original, 
                     c=percentage_errors, cmap='viridis', alpha=0.6, s=30)
ax1.plot([y_test_original.min(), y_test_original.max()], 
         [y_test_original.min(), y_test_original.max()], 'r--', alpha=0.8)
ax1.set_xlabel('Actual Price (USD)')
ax1.set_ylabel('Predicted Price (USD)')
ax1.set_title('Actual vs Predicted Prices\n(Color = Percentage Error)')
plt.colorbar(scatter, ax=ax1, label='Percentage Error (%)')

# Highlight high error points
high_error_actual = y_test_original[high_error_mask]
high_error_pred = y_test_pred_original[high_error_mask]
ax1.scatter(high_error_actual, high_error_pred, color='red', s=50, alpha=0.8, 
           edgecolors='black', linewidth=1, label=f'Error > 20% (n={len(high_error_actual)})')
ax1.legend()

# 2. Distribution of percentage errors
ax2 = axes[0, 1]
ax2.hist(percentage_errors, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(x=20, color='red', linestyle='--', linewidth=2, label='20% threshold')
ax2.axvline(x=percentage_errors.mean(), color='orange', linestyle='-', linewidth=2, 
           label=f'Mean: {percentage_errors.mean():.1f}%')
ax2.set_xlabel('Percentage Error (%)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Percentage Errors')
ax2.legend()

# 3. Error vs Actual Price
ax3 = axes[1, 0]
ax3.scatter(y_test_original, percentage_errors, alpha=0.6, s=30, color='blue')
ax3.scatter(y_test_original[high_error_mask], percentage_errors[high_error_mask], 
           color='red', s=50, alpha=0.8, edgecolors='black', linewidth=1,
           label=f'Error > 20% (n={np.sum(high_error_mask)})')
ax3.axhline(y=20, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax3.set_xlabel('Actual Price (USD)')
ax3.set_ylabel('Percentage Error (%)')
ax3.set_title('Percentage Error vs Actual Price')
ax3.legend()

# 4. Residuals plot
ax4 = axes[1, 1]
residuals = y_test_pred_original - y_test_original
ax4.scatter(y_test_original, residuals, alpha=0.6, s=30, color='blue')
ax4.scatter(y_test_original[high_error_mask], residuals[high_error_mask], 
           color='red', s=50, alpha=0.8, edgecolors='black', linewidth=1,
           label=f'Error > 20% (n={np.sum(high_error_mask)})')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax4.set_xlabel('Actual Price (USD)')
ax4.set_ylabel('Residuals (Predicted - Actual)')
ax4.set_title('Residuals vs Actual Price')
ax4.legend()

plt.tight_layout()
plt.show()

# Show details of worst predictions
print(f"\nTop 10 worst predictions (highest percentage errors):")
print("="*80)
worst_indices = np.argsort(percentage_errors)[-10:][::-1]
for i, idx in enumerate(worst_indices, 1):
    actual = y_test_original[idx]
    predicted = y_test_pred_original[idx]
    error = percentage_errors[idx]
    print(f"{i:2d}. Actual: ${actual:8,.0f} | Predicted: ${predicted:8,.0f} | Error: {error:5.1f}%")